# Spell Checker Program
Welcome to the Spell Checker program! In this program, we will implement an effective and useful spell checking system using three approaches.

## Import libraries

In [11]:
import re
import numpy as np
from collections import Counter

## 1st Approach - Simple Spell Checker:

The Simple spell checker is based on the frequency of words in a corpus. It provides a simple and deterministic approach to spell checking, but its accuracy heavily relies on the quality and coverage of the spelling rules.

#### How it works

1. The spell checker takes a text as input and tokenizes it into sentences.
2. For each sentence, it tokenizes it into words.
3. For each word, it checks if it is a misspelled word (not present in the vocabulary).
4. If the word is misspelled, it generates a list of possible corrections based on a set of predefined spelling rules.
5. It selects the most probable correction based on the probability
6. It replaces the misspelled word with the most probable correction in the corrected text.
7. Finally, it returns the corrected version of the input text.

### Load the data

The `load_data` function loads the corpus data from a file and returns it as a string. It takes two parameters: 
- _file_name_ (name of the file) and 
- _printing_ (a boolean indicating whether to print information about the data). 

The loaded data is returned as a string.

In [12]:
def load_data(file_name: str, printing: bool=False) -> str:
    """
    Load the corpus data from a file and return it as a list of strings.

    Parameters:
        file_name: Name of the file.
        printing: If true, some information about the data will be printed.
    
    Return: 
        str: The loaded data as a string.
    """
    try:
        with open(file_name, "r", encoding='utf-8') as f:
            data = f.read().lower()

        if printing:
            print("Data type:", type(data))
            print(f"Number of letters: {len(data):,d}")
            print("First 100 letters of the data")
            print("-"*30)
            display(data[0:100])
            print("-"*30)

            print("Last 100 letters of the data")
            print("-"*30)
            display(data[-100:])
            print("-"*30)
            
        return data
    
    except FileNotFoundError:
        raise FileNotFoundError(f"Error: File '{file_name}' not found.")

### Prepare the Input String:

`clean_input` function takes an _input string_ and returns a list of lowercase words from the input. It removes non-alphabetic characters from the input string.

In [13]:
def clean_input(input_str: str):
    """
    Input: 
        input_str: a string to check the spell
    Output: 
        cleaned_input: a list of lower case words of input_str, 
          the list dosen't contains any non-alphabetic characters
    """

    pattern = re.sub(r'\W+|\b\d+\b', ' ', input_str)
    cleaned_input = pattern.lower().split()

    return cleaned_input

### Is the word in the corpus?

`is_word` function checks if a given word is present in the vocabulary. It takes two parameters: 
- _word_ (the word to check) 
- _vocabulary_ (the set of words representing the vocabulary). 

It returns True if the word is in the dictionary and False otherwise.

In [14]:
def is_word(word: str, vocabulary: set[str]) -> bool:
    """
    Check if a word is in the vocabulary.

    Parameters:
        word: The word to check.
        vocabulary: The set of words representing the vocabulary.

    Returns:
        bool: True if the word is in the dictionary, False otherwise.
    """
    return word.lower() in vocabulary

### Word Counter:

`count` function counts the _frequency_ of words in a given list and returns a dictionary where the key is the word and the value is its frequency.

In [15]:
def count(word_list: list[str]):
    """
    Count the frequency of words in a list and return a dictionary.

    Parameters:
        word_list: A list of words.

    Returns:
        The wordcount dictionary where key is the word and value is its frequency.
    """
    return Counter(word_list)

### Probability calculation:

`get_probs` function calculates the probability of each word based on its frequency. It takes a _word_count_dict_ as input, which is a dictionary where the key is the word and the value is its frequency. It returns a dictionary _probs_ where keys are the words and the values are the probability that a word will occur.

In [16]:
def get_probs(word_count_dict):
    """
    Calculate the probability of each word based on its frequency.
    
    Parametres:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    
    Returns:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur. 
    """
    
    total_words = len(word_count_dict)
    probs = {word: count/total_words for word, count in word_count_dict.items()}

    return probs

### edit_one function:

`edit_one_v1` function generates a set of possible corrections for a misspelled word with one edit. It takes a _misspelled word_ as input and returns a set of possible corrections.

In [17]:
def edit_one_v1(word: str):
    """
    Generate a set of possible corrections for a misspelled word with one edit.

    Parameters:
        word: The misspelled word.

    Returns:
        set: A set of possible corrections.
    """

    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [left + right[1:] for left, right in splits if right]
    inserts = [left + c + right for left, right in splits for c in alphabet]
    replaces = [left + c + right[1:] for left, right in splits if right for c in alphabet]
    transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right) > 1]
    
    return set(deletes + inserts + replaces + transposes)

### Generate corrections:

`generate_corrections_v1` function generates a set of possible corrections for a misspelled word using the edit distance algorithm. It takes a misspelled word and the number of n_edits as input and returns a set of possible corrections.

In [18]:
def generate_corrections_v1(word: str, n_edits: int):
    """
    Generate a set of possible corrections for a misspelled word using the edit distance algorithm.

    Parameters:
        word The misspelled word.
        n_edits: Number of edits.

    Returns:
        set: A set of possible corrections.
    """
    
    edits = set()

    if n_edits == 1:
        edits = edit_one_v1(word)
    else:
        edit_set = {word}
        for i in range(n_edits):
            edit_iter = set()
            for w in edit_set:
                edit_iter.update(edit_one_v1(w))
            edit_set = edit_iter
        edits = edit_set

    return edits

### Spell Checking:

`spell_check_v1` function checks the spelling of a piece of text. It takes the following parameters:
- text (str): The text to check;
- vocab (list): List of words representing the vocabulary;
- probs (dict): Dictionary of word probabilities;
- n_edits (int): Maximum number of edits allowed;
- n (int): Number of possible word corrections to be returned in the dictionary.

The function returns a list of misspelled words and their possible corrections.

In [19]:
def spell_check_v1(text: str, vocab: list[str], probs: dict, n_edits: int=1, n: int=2):
    """
    Check the spelling of a piece of text.

    Parameters:
        text: The text to check;
        vocab: list of words, vocabulary;
        probs: Dictionary of word probabilities;
        n_edits: maximum number of edits;
        n: number of possible word corrections you want returned in the dictionar.

    Returns:
        list: A list of misspelled words and their possible corrections.
    """

    words = clean_input(text)
    suggestions = []
    n_best = []
    corrected_text = text

    for word in words:
        if not is_word(word, vocab):
            corrections = []
            a = 1
            while not corrections:
                if a > n_edits: break
                corrections = generate_corrections_v1(word, a)
                a += 1
            tmp_suggestions = {c: probs[c] for c in corrections if is_word(c, vocab)}
            suggestions.append((word, tmp_suggestions))

    suggestions_sorted = [[word, dict(sorted(sugg.items(), key=lambda x: x[1], reverse=True))] for word, sugg in suggestions]

    corrected_sentence = text.split()
    for i, (word, sugg) in enumerate(suggestions_sorted):
        top_n_sugg = dict(sorted(sugg.items(), key=lambda x: x[1], reverse=True)[:n])
        n_best.append((word, dict(top_n_sugg)))

        if top_n_sugg:
            best_correction = next(iter(top_n_sugg))
            corrected_sentence[i] = best_correction
            corrected_text = corrected_text.replace(word, best_correction)

    return n_best, corrected_text

### Test first approach:

Now we will test the first approach of the spell checker. First we'll load data from the `en_US_twitter.txt` file, then count unique words, calculate word probabilities, and perform spell checking on a given text. The misspelled words and their possible corrections are printed.

In [20]:
data = load_data('en_US_twitter.txt')
word_list = re.findall('\w+', data)
word_vocab = set(word_list)
word_count_dict = count(word_list)
probs = get_probs(word_count_dict)

print(f"There are {len(word_vocab)} unique words in the vocabulary.")
print(f"The count for the word 'say' is {word_count_dict.get('name',0)}")

There are 36894 unique words in the vocabulary.
The count for the word 'say' is 238


In [21]:
text1 = "In Krav Maga thre are no rulse, no restrctions.\nI actaully lked Derek Morris as a Ranger."
text2 = "Thanks for the quck birhday lessn"

n_best, corrected_sentence = spell_check_v1(text1, word_vocab, probs, n_edits=1, n=2)

In [22]:
print("Original sentence:\n", text1, end='\n'*2)
print("Corrected sentence:\n", corrected_sentence, end='\n'*2)
print("Suggestions for misspelled words:")

for m in n_best:
    print(f"\n  {m[0]} :")
    for w, p in m[1].items():
        print(f"\t{w}: {p}")

Original sentence:
 In Krav Maga thre are no rulse, no restrctions.
I actaully lked Derek Morris as a Ranger.

Corrected sentence:
 In Krav Maga the are no rules, no restrictions.
I actaully liked Derek Morris as a Ranger.

Suggestions for misspelled words:

  thre :
	the: 0.5175909361955874
	there: 0.039112050739957716

  rulse :
	rules: 0.0010028730958963517
	rule: 0.0008131403480240689

  restrctions :
	restrictions: 5.420935653493793e-05

  lked :
	liked: 0.001599176017780669
	led: 0.0003794654957445655



## 2nd Approach - Probability-Based Spell Checker


The probability-based spell checker leverages a language model to estimate the likelihood of different corrections for misspelled words. By considering the context of the surrounding words, it provides more accurate and context-aware spell corrections compared to the Simple approach. However, it requires a large and diverse training corpus to train the language model effectively.


#### How it works

1. The spell checker takes a text as input and tokenizes it into sentences.
2. For each sentence, it tokenizes it into words.
3. For each word, it checks if it is a misspelled word (not present in the vocabulary).
4. If the word is misspelled, it generates a list of possible corrections based on the Levenshtein distance algorithm.
5. It calculates the probabilities of each correction using a language model trained on a large corpus of text.
6. It selects the most probable correction based on the calculated probabilities.
7. It replaces the misspelled word with the most probable correction in the corrected text.
8. Finally, it returns the corrected version of the input text.

### Split data by linebreak "\n":

`split_to_sentences` function takes the input data as a string and splits it into individual sentences based on line breaks (`"\n"`). It returns a list of sentences.

In [23]:
def split_to_sentences(data: str) -> list[str]:
    """
    Split data by linebreak "\n"
    
    Parameters
        data: The input data as a string.
    
    Returns:
        list: A list of sentences
    """
    sentences = data.split('\n')
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    
    return sentences 

### Tokenizes a sentence into tokens:

`tokenize_sentence` function takes a sentence as input, converts it to lowercase, removes punctuation, and splits it into individual tokens. It returns a list of tokens.

In [24]:
def tokenize_sentence(sentence: str) -> list[str]:
    """
    Tokenize a sentence into tokens.

    Parameters:
        data: The input sentence.
        
    Returns:
        List: A list of tokens.
    """
    sentence = sentence.lower()
    sentence = re.sub(r"[^\w\s]", "", sentence)
    tokens = sentence.split()
    return tokens

### List of tokenized sentences:

`get_tokenized_data` function takes the input data as a string, splits it into sentences using the previous function, and tokenizes each sentence using the previous function. It returns a tuple containing the list of sentences and the list of tokenized sentences.

In [25]:
def get_tokenized_data(data: str) -> tuple[list[str], list[list[str]]]:
    """
    Tokenize the data into sentences and words.

    Parameters:
        data: The input data as a string.

    Returns:
        tuple: A tuple containing the list of sentences and list of tokenized sentences.
    """
    sentences = split_to_sentences(data)
    tokenized_sentences = [tokenize_sentence(s) for s in sentences]
    
    return sentences, tokenized_sentences

### Vocabulary:

`get_vocabulary` function takes the tokenized data (list of lists of tokens) and extracts the unique words to create a vocabulary. It returns a set containing the unique words.

In [26]:
def get_vocabulary(tokenized_data: list[list[str]]) -> set[str]:
    """
    Extract the vocabulary from a list of tokenized data.

    Parameters:
        tokenized_data: A list of tokenized sentences.

    Returns:
        set: A set containing the unique words present in the tokenized data.
    """
    vocab = set()
    for sublist in tokenized_data:
        for item in sublist:
            vocab.add(item)
    return vocab

### Split data:

`split_data` function splits the tokenized data into training and test sets based on the specified train ratio. It returns a tuple containing the train and test sets.

In [27]:
def split_data(tokenized_data: list[list[str]], train_ratio: float=0.8,
                printing: bool=False) -> tuple[list[list[str]], list[list[str]]]:
    """
    Splits the data into train and test sets based on the specified ratio.
    
    Parameters:
        tokenized_data: The input data as a list of lists of tokens.
        train_ratio: The ratio of data to use for training (between 0 and 1).
        printing: If true, information about the split will be printed.
        
    Returns:
        Tuple: Train and test sets as lists of sentences or sequences.
    """
    train_size = int(len(tokenized_data) * train_ratio)
    train_data = tokenized_data[:train_size]
    test_data = tokenized_data[train_size:]

    if printing:
        print(f"{len(tokenized_data)} data are split into {len(train_data)} train and {len(test_data)} test set")

        print("First training sample:")
        print(train_data[0])
            
        print("First test sample")
        print(test_data[0])
    
    return train_data, test_data

### Minimum Edit Distance:

`calculate_edit_distance` function calculates the minimum edit distance between two words using the MED algorithm. It takes the source and target words, along with optional insertion, deletion, and replacement costs, and returns the minimum edit distance matrix and the minimum edit distance value.

In [28]:
def calculate_edit_distance(source: str, target: str, ins_cost: int= 1, 
                            del_cost: int= 1, rep_cost: int= 2) -> tuple[np.ndarray, int]:
    """
    Calculate the minimum edit distance between two words.
    
    Parameters:
        source: The source word.
        target: The target word.
        ins_cost: The cost of insertion (default is 1).
        del_cost: The cost of deletion (default is 1).
        rep_cost: The cost of replacement (default is 2).

    Returns:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    """
    m, n = len(source), len(target) 
    D = np.zeros((m+1, n+1), dtype=int) 
    
    for row in range(1,m+1):
        D[row,0] = D[row-1,0] + del_cost
        
    for col in range(1,n+1):
        D[0,col] = D[0,col-1] + ins_cost
        
    for row in range(1,m+1): 
        for col in range(1,n+1):
            r_cost = rep_cost
            if source[row-1] == target[col-1]:
                r_cost = 0
            D[row,col] = min(D[row-1,col] + del_cost, D[row,col-1] + ins_cost, D[row-1,col-1] + r_cost)
          
    med = D[m,n]
    
    return D, med

### Implement one_edit:

`edit_one` function generates a set of possible corrections for a misspelled word with one edit. It takes the misspelled word and the vocabulary as input and returns a set of possible corrections.

In [29]:
def edit_one(word: str, vocabulary: list[str]) -> set[str]:
    """
    Generate a set of possible corrections for a misspelled word with one edit.

    Parameters:
        word: The misspelled word.
        vocabulary: A list of vocabulary words.

    Returns:
        set: A set of possible corrections.
    """

    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]

    deletes = [left + right[1:] for left, right in splits if right]
    deletes = [d for d in deletes if is_word(d, vocabulary)]

    inserts = [left + c + right for left, right in splits for c in alphabet]
    inserts = [i for i in inserts if is_word(i, vocabulary)]

    replaces = [left + c + right[1:] for left, right in splits if right for c in alphabet]
    replaces = [r for r in replaces if is_word(r, vocabulary)]

    transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right) > 1]
    transposes = [t for t in transposes if is_word(t, vocabulary)]
    
    return set(deletes + inserts + replaces + transposes)

### Generate possible corrections:

`get_corrections` function generates a list of possible corrections for a misspelled word based on the given vocabulary and maximum edit distance. It takes the misspelled word, vocabulary, number of edits allowed, and maximum edit distance as input and returns a set of possible corrections.

In [30]:
def get_corrections(word: str, vocabulary: list[str], n_edits: int=1, 
                    max_distance: int=2) -> set[str]:
    """
    Generate a list of possible corrections for a misspelled word based on the given vocabulary and maximum edit distance.

    Parameters:
        word: The misspelled word.
        vocabulary: A list of vocabulary words.
        n_edits: The number of edits allowed to generate corrections (default is 1).
        max_distance: The maximum edit distance allowed for a correction to be considered (default is 2).

    Returns:
        possible_corrections: A set of possible corrections.
    """
    possible_corrections = set()

    if n_edits == 1:
        possible_corrections = {corr for corr in edit_one(word, vocabulary) if calculate_edit_distance(word, corr)[1]<=max_distance}
    else:
        previous_edits = {word}
        for _ in range(n_edits):
            current_edits = set()
            for prev_edit in previous_edits:
                new_corrections = {corr for corr in edit_one(prev_edit, vocabulary) if calculate_edit_distance(word, corr)[1]<=max_distance}
                current_edits.update(new_corrections)
            previous_edits = current_edits
        possible_corrections = previous_edits
        
    possible_corrections = sorted(possible_corrections, key=lambda corr: calculate_edit_distance(word, corr)[1])

    return possible_corrections

### Count words:

`count_words` function counts the number of word appearances in the tokenized sentences. It takes the tokenized sentences as input and returns a dictionary mapping each word to its frequency.

In [31]:
def count_words(tokenized_sentences: list[list[str]]) -> dict[str, int]:
    """
    Count the number of word appearence in the tokenized sentences.
    
    Parameters:
        tokenized_sentences: List of lists of strings.
    
    Returns:
        word_counts: dict that maps word (str) to the frequency (int).
    """
        
    word_counts = {}
    for sentence in tokenized_sentences:
        for token in sentence:
            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1
    
    return word_counts

### Words with n+ frequency:

`get_words_with_nplus_frequency` function finds the words that appear N times or more in the tokenized sentences. It takes the tokenized sentences and the frequency threshold as input and returns a list of words that meet the frequency criteria.

In [32]:
def get_words_with_nplus_frequency(tokenized_sentences: list[list[str]], 
                                   freq_threshold: int) -> list[str]:
    """
    Find the words that appear N times or more.

    Parameters:
        tokenized_sentences: List of lists of sentences.
        freq_threshold: Minimum number of occurrences for a word to be in the closed vocabulary.
    
    Returns:
        closed_vocab: List of words that appear N times or more.
    """
    closed_vocab = []
    word_counts = count_words(tokenized_sentences)
    for word, cnt in word_counts.items():
        if cnt >= freq_threshold:
            closed_vocab.append(word)
    
    return closed_vocab

### Replace OOV words by <unk>:

`replace_oov_words_by_unk` function replaces words not in the given vocabulary with the unknown token (`"<unk>"`). It takes the tokenized sentences, vocabulary, and unknown token as input and returns the tokenized sentences with replaced words.

In [33]:
def replace_oov_words_by_unk(tokenized_sentences: list[list[str]], 
                             vocabulary: list[str], unknown_token: str="<unk>"
                             ) -> list[list[str]]:
    """
    Replace words not in the given vocabulary with the unknown token.

    Parameters:
        tokenized_sentences: List of lists of strings
        vocabulary: List of strings that we will use
        unknown_token: A string representing unknown (out-of-vocabulary) words
    
    Returns:
        replaced_tokenized_sentences: List of lists of strings, with words not in the vocabulary replaced
    """
    
    vocabulary = set(vocabulary)
    replaced_tokenized_sentences = []
    for sentence in tokenized_sentences:
        replaced_sentence = []
        for token in sentence:
            if token in vocabulary:
                replaced_sentence.append(token)
            else:
                replaced_sentence.append(unknown_token)
        replaced_tokenized_sentences.append(replaced_sentence)
        
    return replaced_tokenized_sentences

### Data with \<unk\>:

`data_with_unk` function preprocesses the data by replacing low-frequency words with the unknown token. It takes the tokenized data and frequency threshold as input and returns the data with low-frequency words replaced by `"<unk>"` and the vocabulary of words that appear N times or more in the training data.

In [34]:
def data_with_unk(data: list[list[str]], freq_threshold: int):
    """
    Preprocesses the data by replacing low-frequency words with the unknown token.      
    
    Parameters:
        data: List of lists of strings.
        freq_threshold: Words whose count is less than this are treated as unknown.

    Returns:
        Tuple of
        - data with low frequent words replaced by "<unk>"
        - vocabulary of words that appear n times or more in the training data
    """
   
    vocabulary = get_words_with_nplus_frequency(data, freq_threshold)
    data_replaced = replace_oov_words_by_unk(data, vocabulary)
    
    return data_replaced, vocabulary

### Count n-grams:

`count_n_grams` function counts all n-grams in the given data. It takes the tokenized data, n (number of words in a sequence), start token, and end token as input and returns a dictionary mapping each n-gram to its frequency.

In [35]:
def count_n_grams(data: list[list[str]], n: int=2, 
                  start_token: str='<s>', end_token: str= '<e>'
                  ) -> dict:
    """
    Count all n-grams in the given data
    
    Parameters:
        data: List of lists of words.
        n: number of words in a sequence (default is 2).
        start_token: a string indicate the beginning of the sentence (default is '<s>').
        end_token: a string indicate the end of the sentence (default is '<e>').
    
    Returns:
        n_grams: A dictionary that maps a tuple of n-words to its frequency
    """
    n_grams = {}

    for sentence in data:
        sentence = [start_token]*n + sentence + [end_token]
        sentence = tuple(sentence)
        m = len(sentence) if n==1 else len(sentence)-1
        for i in range(m): 
            n_gram = sentence[i:i+n]
            if n_gram in n_grams.keys():
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1
    
    return n_grams

### Estimate probability:

`estimate_probability` function estimates the probabilities of a next word using the n-gram counts with k-smoothing. It takes the next word, previous n-gram, n-gram counts, (n+1)-gram counts, vocabulary size, and smoothing parameter as input and returns the probability.

In [36]:
def estimate_probability(word: str, previous_n_gram: list[str], 
                         n_gram_counts: dict, n_plus1_gram_counts: dict, 
                         vocabulary_size: int, k: float=1.0) -> float:
    """
    Estimate the probabilities of a next word using the n-gram counts with k-smoothing
    
    Parameters:
        word: Next word
        previous_n_gram: A sequence of words of length n
        n_gram_counts: Dictionary of counts of n-grams
        n_plus1_gram_counts: Dictionary of counts of (n+1)-grams
        vocabulary_size: Number of words in the vocabulary
        k: Positive constant, smoothing parameter
    
    Returns:
        A probability
    """
    previous_n_gram = tuple(previous_n_gram)
    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts  else 0
    denominator = previous_n_gram_count + k * vocabulary_size

    n_plus1_gram = previous_n_gram + (word,)
    n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts  else 0
    numerator = n_plus1_gram_count + k
    
    probability = numerator / denominator
        
    return probability

### Spell Checking:

This function performs spell checking on the given text using an n-gram language model. It takes the text, vocabulary, top N suggestions to consider, order of the n-gram language model, smoothing parameter, maximum number of edits allowed in a suggested correction, and.

In [37]:
def spell_check_v2(text: str, vocabulary: set[str], top_n: int=2, n_g: int=2,k: int=1.0, 
                n_edits: int=1, max_distance: int=2) -> tuple[dict, str]:
    """
    Perform spell checking on the given text using an n-gram language model.

    Parameters:
        text: The text to perform spell checking on.
        vocabulary: A set of words representing the vocabulary.
        top_n: The number of top suggestions to consider (default is 2).
        n_g: The order of the n-gram language model (default is 2).
        k: Positive constant, smoothing parameter (default is 1.0).
        n_edits: The maximum number of edits allowed in a suggested correction (default is 1).
        max_distance: The maximum edit distance allowed for a suggested correction (default is 2).

    Returns:
        sorted_dict : Dictionary of suggestions.
        corrected_text: The corrected version of the input text.
    """
    suggestions = dict()
    _, tokenized_sentences = get_tokenized_data(text)
    n_grams = count_n_grams(tokenized_sentences, n_g)
    n_plus1_grams = count_n_grams(tokenized_sentences, n_g+1)
    corrected_text = text
    
    for sentence in tokenized_sentences:
        index = None
        tmp_sentence = ['<s>']*n_g + sentence + ['<e>']
        
        for token in sentence:
            probas = dict()
            if not is_word(token, vocabulary):
                index = tmp_sentence.index(token)
                previous_n_gram = tuple(tmp_sentence[abs(index-n_g):index])
                corrections = get_corrections(token, vocabulary, n_edits, max_distance)
                corrections = [c for c in corrections if is_word(c, vocabulary)]
                for corr in corrections:
                    proba = estimate_probability(corr, previous_n_gram, n_grams,
                                                 n_plus1_grams, len(vocabulary), k)
                    probas[corr] = proba
                suggestions[token] = probas
        
        sorted_suggestions = {k: dict(sorted(v.items(), key=lambda item: item[1], reverse=True)) for k, v in suggestions.items()}
        sorted_dict = {}
        for key, inner_dict in sorted_suggestions.items():
            sorted_inner_dict = dict(sorted(inner_dict.items(), key=lambda item: item[1], reverse=True)[:top_n])
            sorted_dict[key] = sorted_inner_dict

        for key in sorted_dict.keys():
            if key in corrected_text:
                first_inner_key = next(iter(sorted_dict[key]))
                corrected_text = corrected_text.replace(key, first_inner_key)

    return sorted_dict, corrected_text

print("done!")

done!


### Test the code:

In [38]:
data = load_data('en_US_twitter.txt')
sentences, tokenized_data = get_tokenized_data(data)
train, test = split_data(tokenized_data, 0.8)
vocabulary = get_vocabulary(tokenized_data)

In [39]:
text1 = "In Krav Maga thre are no rulse, no restrctions.\nI actaully lked Derek Morris as a Ranger."
text2 = "Thanks for the quck birhday lessn"

sorted_dict, corrected_text = spell_check_v2(text1, vocabulary)

In [40]:
print(f"Original text:\n{text1}\n")
print(f"Corrected text:\n{corrected_text}\n")
print(f"The misspelled words and thier corrections:")
for word in sorted_dict.keys():
    print('-'*50)
    print(f"  {word}:")
    for c, p in sorted_dict[word].items():
        print(f"    {c}:\t{p}")

Original text:
In Krav Maga thre are no rulse, no restrctions.
I actaully lked Derek Morris as a Ranger.

Corrected text:
In Krav Maga threww are no rule, no restrictions.
I actaully led Derek Morris as a Ranger.

The misspelled words and thier corrections:
--------------------------------------------------
  thre:
    threw:	2.3643456673365648e-05
    thr:	2.3643456673365648e-05
--------------------------------------------------
  rulse:
    rule:	2.3643456673365648e-05
    ruse:	2.3643456673365648e-05
--------------------------------------------------
  restrctions:
    restrictions:	2.3643456673365648e-05
--------------------------------------------------
  lked:
    led:	2.3643456673365648e-05
    liked:	2.3643456673365648e-05


## 3rd Approach - Sequence-Based Spell Checker:

This code implements a sequence-based spell checker using an n-gram language model. The spell checker analyzes the input text, identifies misspelled words, and suggests corrections based on the context of the surrounding words.

#### How it works

1. The function tokenizes the text into sentences and tokenizes each sentence into words.
2. For each word in the tokenized text, it checks if it is a misspelled word (not present in the vocabulary).
3. If the word is misspelled, it identifies the surrounding context by extracting the n-gram sequence.
4. It generates a list of possible corrections for the misspelled word based on the maximum number of edits and maximum edit distance allowed.
5. It calculates the probabilities of each correction using the n-gram language model.
6. It selects the top-n corrections with the highest probabilities.
7. It replaces the misspelled word with the most probable correction in the corrected text.
8. Finally, it returns the dictionary of suggestions and the corrected version of the input text.

### List of squences:

`get_sequences` function takes a sentence and generates sequences of a specified length. It returns a list of sequences.

In [41]:
def get_sequences(sentence: str, lenght:int = 2) -> list[str]:
    """
    Get the sequences of a sentence.

    Parameters:
        sentence: a string.
        lenght: the length of the sequence.

    Returns:
        lis: a list of sequences.
    """
    sequences = [sentence[i:i+lenght] for i in range(0, len(sentence)-lenght)]
    return sequences

### Count sequences:

`count_sequences` function counts the number of occurrences of each sequence in the tokenized sentences. It takes a dataset and the length of the sequence as input, and returns a dictionary that maps each sequence to its frequency.

In [42]:
def count_sequences(data: str, lenght: int=2) -> dict[str, int]:
    """
    Count the number of word appearence in the tokenized sentences.
    
    Parameters:
        data: dataset, large text(s).
        lenght: the lenght of the sequence.
    
    Returns:
        sequence_counts: dict that maps sequence (str) to the frequency (int).
    """
        
    sequence_counts = {}
    for i in range(lenght, len(data), lenght):
        sequence = data[i-lenght, i]
        if sequence not in sequence_counts.keys():
            sequence_counts[sequence] = 1
        else:
            sequence_counts[sequence] += 1
    
    return sequence_counts

### Count n-grams:

`count_n_grams_v2` function counts all n-grams in the given data. It takes a list of sentences, the order of the n-gram language model, the length of the sequence, and optional start and end tokens as input. It returns a dictionary that maps each n-gram to its frequency.

In [43]:
def count_n_grams_v2(sentences: list[str], n: int=2, 
                  lenght: int = 2, start_token: str='ð', 
                  end_token: str= '§') -> dict[str, int]:
    """
    Count all n-grams in the given data
    
    Parameters:
        data: List of words.
        n: number of words in a sequence (default is 2).
        start_token: a string indicate the beginning of the sentence (default is 'ð').
        end_token: a string indicate the end of the sentence (default is '§').
    
    Returns:
        n_grams: A dictionary that maps a tuple of n-words to its frequency
    """
    n_grams = {}

    for sentence in sentences:
        sentence = start_token*n + sentence + end_token
        sequences = get_sequences(sentence, lenght)
        m = len(sequences) if n==1 else len(sequences)-1
        for i in range(m): 
            n_gram = tuple(sequences[i:i+n])
            if n_gram in n_grams.keys():
                n_grams[tuple(n_gram)] += 1
            else:
                n_grams[tuple(n_gram)] = 1
    
    return n_grams

### Estimate probability

`estimate_probability_v2` function estimates the probabilities of the next word using the n-gram counts with k-smoothing. It takes a sequence, the previous n-gram, the dictionaries of n-gram and (n+1)-gram counts, the number of sequences in the dataset, and a smoothing parameter as input. It calculates the probability based on the counts and returns the estimated probability.

In [44]:
def estimate_probability_v2(sequence: str, previous_n_gram: list[str],
                          n_gram_counts: dict, n_plus1_gram_counts: dict, 
                          num_seq: int, k: float=1.0) -> float:
    """
    Estimate the probabilities of a next word using the n-gram counts with k-smoothing
    
    Parameters:
        sequence: Next sequence
        previous_n_gram: A sequence of words of length n
        n_gram_counts: Dictionary of counts of n-grams
        n_plus1_gram_counts: Dictionary of counts of (n+1)-grams
        num_seq: Number of sequences in the dataset
        k: Positive constant, smoothing parameter
    
    Returns:
        A probability
    """
    previous_n_gram = tuple(previous_n_gram)
    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
    denominator = previous_n_gram_count + k * num_seq

    n_plus1_gram = previous_n_gram + (sequence,)
    n_plus1_gram_count = n_plus1_gram_counts.gets(n_plus1_gram,0) if n_plus1_gram[0] in n_plus1_gram_counts else 0
    numerator = n_plus1_gram_count + k
    
    probability = numerator / denominator
        
    return probability

### Spell Checking

`spell_check_v3` function performs spell checking on the given text using the n-gram language model. It takes the input text, a vocabulary set, and various parameters such as the number of top suggestions, the order of the n-gram language model, the smoothing parameter, the maximum number of edits allowed, and the maximum edit distance allowed. It returns a dictionary of suggestions and the corrected version of the input text.

In [45]:
def spell_check_v3(text, vocabulary, top_n=2, n_g=2, k=1.0,
                length=2, n_edits=1, max_distance=2):
    """
    Perform spell checking on the given text using an n-gram language model.

    Parameters:
        text: The text to perform spell checking on.
        vocabulary: A set of words representing the vocabulary.
        top_n: The number of top suggestions to consider (default is 2).
        n_g: The order of the n-gram language model (default is 2).
        k: Positive constant, smoothing parameter (default is 1.0).
        n_edits: The maximum number of edits allowed in a suggested correction (default is 1).
        max_distance: The maximum edit distance allowed for a suggested correction (default is 2).

    Returns:
        sorted_dict: Dictionary of suggestions.
        corrected_text: The corrected version of the input text.
    """
    suggestions = {}
    sentences, tokenized_sentences = get_tokenized_data(text)
    n_grams = count_n_grams_v2(sentences, n=n_g, lenght=length)
    n_plus1_grams = count_n_grams_v2(sentences, n=n_g+1, lenght=length)
    corrected_text = text
    
    for sentence, tokenize_sentence in zip(sentences, tokenized_sentences):
        index = None
        tmp_sentence = 'ð'*n_g + sentence + '§'
        tmp_tokenize_sentence = ['ð']*n_g + tokenize_sentence + ['§']
        sequences = get_sequences(tmp_sentence, length)
        for token in tokenize_sentence:
            probas = {}
            if not is_word(token, vocabulary):
                index = sequences.index(token[0:length])
                previous_n_gram = tuple(sequences[max(index-n_g, 0):index])
                corrections = get_corrections(token, vocabulary, n_edits, max_distance)
                corrections = [c for c in corrections if is_word(c, vocabulary)]
                for corr in corrections:
                    corr_seq = get_sequences(corr, length)
                    proba = estimate_probability_v2(corr_seq, previous_n_gram, n_grams,
                                                 n_plus1_grams, len(vocabulary), k)
                    probas[corr] = proba
                suggestions[token] = probas
        
        sorted_suggestions = {k: dict(sorted(v.items(), key=lambda item: item[1], reverse=True)) for k, v in suggestions.items()}
        sorted_dict = {}
        for key, inner_dict in sorted_suggestions.items():
            sorted_inner_dict = dict(sorted(inner_dict.items(), key=lambda item: item[1], reverse=True)[:top_n])
            sorted_dict[key] = sorted_inner_dict

        for key in sorted_dict.keys():
            if key in corrected_text:
                first_inner_key = next(iter(sorted_dict[key]))
                corrected_text = re.sub(r'\b' + re.escape(key) + r'\b', first_inner_key, corrected_text)

    return sorted_dict, corrected_text

print("done!")

done!


### Test the code:

In [46]:
data = load_data('en_US_twitter.txt', printing=False)
sentences, tokenized_data = get_tokenized_data(data)
train, test = split_data(tokenized_data, 0.8)
vocabulary = get_vocabulary(tokenized_data)
length=2
n_g = 2

In [47]:
text = "In Krav Maga thre are no rulse, no restrctions.\nI actaully lked Derek Morris as a Ranger."
text2 = "Thanks for the quck birhday lessn"

sorted_dict, corrected_text = spell_check_v3(text1, vocabulary)

In [48]:
print(f"Original text:\n{text1}\n")
print(f"Corrected text:\n{corrected_text}\n")
print(f"The misspelled words and thier corrections:")
for word in sorted_dict.keys():
    print('-'*50)
    print(f"  {word}:")
    for c, p in sorted_dict[word].items():
        print(f"    {c}:\t{p}")

Original text:
In Krav Maga thre are no rulse, no restrctions.
I actaully lked Derek Morris as a Ranger.

Corrected text:
In Krav Maga threw are no rule, no restrictions.
I actaully led Derek Morris as a Ranger.

The misspelled words and thier corrections:
--------------------------------------------------
  thre:
    threw:	2.3643456673365648e-05
    thr:	2.3643456673365648e-05
--------------------------------------------------
  rulse:
    rule:	2.364289767353887e-05
    ruse:	2.364289767353887e-05
--------------------------------------------------
  restrctions:
    restrictions:	2.3643456673365648e-05
--------------------------------------------------
  lked:
    led:	2.3643456673365648e-05
    liked:	2.3643456673365648e-05
